In [92]:
import numpy as np
import os
import pickle as p
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import networkx as nx
import sys
import slam.io as sio
import networkx as nx
import tools.graph_visu as gv
import tools.graph_processing as gp
import matplotlib.pyplot as plt
import random
import plotly.express as px
import plotly.figure_factory as ff
import trimesh
import slam.topology as stop
from sphere import *
import random

In [13]:
def generate_sphere_random_sampling(vertex_number=100, radius=1.0):
    """
    generate a sphere with random sampling
    :param vertex_number: number of vertices in the output spherical mesh
    :param radius: radius of the output sphere
    :return:
    """
    coords = np.zeros((vertex_number, 3))
    for i in range(vertex_number):
        M = np.random.normal(size=(3, 3))
        Q, R = np.linalg.qr(M)
        coords[i, :] = Q[:, 0].transpose() * np.sign(R[0, 0])
    if radius != 1:
        coords = radius * coords
    return coords

In [12]:
def tri_from_hull(vertices):
    """
    compute faces from vertices using trimesh convex hull
    :param vertices: (n, 3) float
    :return:
    """
    mesh = trimesh.Trimesh(vertices=vertices, process=False)
    return mesh.convex_hull

In [76]:
def generate_noisy_graph(original_graph, nb_vertices, sigma_noise_nodes = 1, sigma_noise_edges = 1, radius = 100):
    
    # generate ground_truth permutation
    ground_truth_permutation = np.random.permutation(nb_vertices)
    #ground_truth_permutation = np.arange(nb_vertices)
    
    # create a new graph
    noisy_graph = nx.Graph()

    # add the nodes (not very clean but it works fine and run in no time)
    for node_to_add in range(len(ground_truth_permutation)):
        for original_node, current_node in enumerate(ground_truth_permutation):
            if current_node == node_to_add:
                print(original_node,node_to_add)
                noisy_coordinate = original_graph.nodes[original_node]["coord"] + \
                    np.random.multivariate_normal(np.zeros(3), np.eye(3) * sigma_noise_nodes)

                # We project on the sphere
                noisy_coordinate = noisy_coordinate / np.linalg.norm(noisy_coordinate) * radius
                noisy_graph.add_node(node_to_add, coord = noisy_coordinate)
        
    compute_noisy_edges = tri_from_hull(list(nx.get_node_attributes(noisy_graph,'coord').values())) # take all peturbated coord and comp conv hull.
    adja = stop.edges_to_adjacency_matrix(compute_noisy_edges) # compute the new adjacency mat
    edge_list = [tuple((u,v)) for u,v in zip(adja.nonzero()[0],adja.nonzero()[1])] # convert to edge list to add edge attributes.


    # add the edges
    for edge in edge_list:

        # get the original and corresponding ends
        #end_a_corresponding, end_b_corresponding = ground_truth_permutation[edge[0]], ground_truth_permutation[edge[1]]
        coordinate_a, coordinate_b = noisy_graph.nodes[edge[0]]["coord"], noisy_graph.nodes[edge[1]]["coord"]

        # calculate noisy geodesic distance
        noisy_geodesic_dist = gp.compute_geodesic_distance_sphere(coordinate_a, coordinate_b, radius)

        # Add the new edge to the graph
        noisy_graph.add_edge(edge[0],edge[1], weight = 1.0, geodesic_distance = noisy_geodesic_dist)
    
    return ground_truth_permutation, noisy_graph

In [82]:
def generate_noisy_graph_2(original_graph, nb_vertices,nb_outliers,sigma_noise_nodes=1, sigma_noise_edges=1, radius=100):
    # Perturbate the coordinates
    # noisy_coord = [points+np.random.multivariate_normal(np.zeros(3), np.eye(3) * sigma_noise_nodes)
    # 	for points in list(nx.get_node_attributes(original_graph,'coord').values())]

    ground_truth_permutation = np.arange(nb_vertices)
    # create a new graph
    noisy_graph = nx.Graph()
    # add the nodes (not very clean but it works fine and run in no time)
    for node_to_add in range(len(ground_truth_permutation)):
        for original_node, current_node in enumerate(ground_truth_permutation):
            if current_node == node_to_add:
                # noisy_coordinate = original_graph.nodes[original_node]["coord"] + \
                # 	np.random.multivariate_normal(np.zeros(3), np.eye(3) * sigma_noise_nodes)

                # Sampling from Von Mises - Fisher distribution
                original_coord = original_graph.nodes[original_node]["coord"]
                mean_original = original_coord / np.linalg.norm(original_coord)  # convert to mean vector
                noisy_coordinate = Sphere().sample(1, distribution='vMF', mu=mean_original,
                                                   kappa=sigma_noise_nodes).sample[0]

                #noisy_coordinate = noisy_coordinate / np.linalg.norm(noisy_coordinate) * radius
                noisy_coordinate  = noisy_coordinate * np.linalg.norm(original_coord)
                noisy_graph.add_node(node_to_add, coord=noisy_coordinate)
                
    noisy_coord = list(nx.get_node_attributes(noisy_graph, 'coord').values())
                
    # Add Outliers
    if nb_outliers > 0:
        print("nb_outliers: ",nb_outliers)
        sphere_random_sampling = generate_sphere_random_sampling(vertex_number=nb_outliers, radius=radius)
        # merge pertubated and outlier coordinates to add edges 
        all_coord = noisy_coord + list(sphere_random_sampling)
    else:
        all_coord = noisy_coord

    print("nb_outliers: ",nb_outliers)


    compute_noisy_edges = tri_from_hull(all_coord)  # take all peturbated coord and comp conv hull.
    adja = stop.edges_to_adjacency_matrix(compute_noisy_edges)  # compute the new adjacency mat.

    # Extracting the ground-truth correspondence
    ground_truth_permutation = []
    for ar1 in compute_noisy_edges.vertices.view(np.ndarray):
        index = 0
        for ar2 in noisy_coord:
            if np.mean(ar1) == np.mean(ar2):
                ground_truth_permutation.append(index)
                index += 1
                break
            else:
                index += 1
                continue

    print("Total Ground truth nodes: ", len(ground_truth_permutation))
    print("Total number of nodes : ", len(compute_noisy_edges.vertices))

    noisy_graph = nx.from_numpy_matrix(adja.todense())

    node_attribute_dict = {}
    for node in noisy_graph.nodes():
        node_attribute_dict[node] = {"coord": np.array(compute_noisy_edges.vertices[node])}

    nx.set_node_attributes(noisy_graph, node_attribute_dict)

    nx.set_edge_attributes(noisy_graph, 1.0, name="weight")

    edge_attribute_dict = {}
    id_counter = 0  # useful for affinity matrix caculation
    for edge in noisy_graph.edges:
        # We calculate the geodesic distance
        end_a = noisy_graph.nodes()[edge[0]]["coord"]
        end_b = noisy_graph.nodes()[edge[1]]["coord"]
        geodesic_dist = gp.compute_geodesic_distance_sphere(end_a, end_b, radius)

        # add the information in the dictionnary
        edge_attribute_dict[edge] = {"geodesic_distance": geodesic_dist, "id": id_counter}
        id_counter += 1

    # add the edge attributes to the graph
    nx.set_edge_attributes(noisy_graph, edge_attribute_dict)

    return np.array(ground_truth_permutation), noisy_graph

In [156]:
def generate_noisy_graph_3(original_graph, nb_vertices,nb_outliers,sigma_noise_nodes=1, sigma_noise_edges=1, radius=100):
    # Perturbate the coordinates
    
    noisy_coord = []
    
    for index in range(nb_vertices):
        
        # Sampling from Von Mises - Fisher distribution
        original_coord = original_graph.nodes[index]["coord"]
        mean_original = original_coord / np.linalg.norm(original_coord)  # convert to mean vector
        noisy_coordinate = Sphere().sample(1, distribution='vMF', mu=mean_original,
                                           kappa=sigma_noise_nodes).sample[0]

        noisy_coordinate  = noisy_coordinate * np.linalg.norm(original_coord)
        #print(noisy_coordinate)
        noisy_coord.append(noisy_coordinate)
                   
    # Add Outliers
    if nb_outliers > 0:
        print("nb_outliers: ",nb_outliers)
        sphere_random_sampling = generate_sphere_random_sampling(vertex_number=nb_outliers, radius=radius)
        # merge pertubated and outlier coordinates to add edges 
        all_coord = noisy_coord + list(sphere_random_sampling)
    else:
        all_coord = noisy_coord

    print("nb_outliers: ",nb_outliers)

    noisy_graph = nx.Graph()

    compute_noisy_edges = tri_from_hull(all_coord)  # take all peturbated coord and comp conv hull.
    adja = stop.edges_to_adjacency_matrix(compute_noisy_edges)  # compute the new adjacency mat.
    

    # Extracting the ground-truth correspondence

    noisy_graph = nx.from_numpy_matrix(adja.todense())

    node_attribute_dict = {}
    for node in noisy_graph.nodes():
        node_attribute_dict[node] = {"coord": np.array(compute_noisy_edges.vertices[node])}

    nx.set_node_attributes(noisy_graph, node_attribute_dict)

    nx.set_edge_attributes(noisy_graph, 1.0, name="weight")

    edge_attribute_dict = {}
    id_counter = 0  # useful for affinity matrix caculation
    for edge in noisy_graph.edges:
        # We calculate the geodesic distance
        end_a = noisy_graph.nodes()[edge[0]]["coord"]
        end_b = noisy_graph.nodes()[edge[1]]["coord"]
        geodesic_dist = gp.compute_geodesic_distance_sphere(end_a, end_b, radius)

        # add the information in the dictionnary
        edge_attribute_dict[edge] = {"geodesic_distance": geodesic_dist, "id": id_counter}
        id_counter += 1

    # add the edge attributes to the graph
    nx.set_edge_attributes(noisy_graph, edge_attribute_dict)

    return noisy_graph,noisy_coord

In [232]:
G = nx.read_gpickle("reference_0.gpickle")

In [237]:
original_coord =  list(nx.get_node_attributes(G,'coord').values())

In [234]:
ground_truth_permutation_old, noisy_graph_old = generate_noisy_graph(G,len(original_coord),10,0)

9 0
3 1
13 2
11 3
4 4
1 5
2 6
19 7
10 8
7 9
16 10
12 11
18 12
15 13
6 14
0 15
5 16
14 17
17 18
8 19


In [81]:
ground_truth_permutation_old

array([17, 11, 13,  3,  7, 18, 10, 14,  1, 16, 19,  2,  0,  6, 15,  5,  8,
        4,  9, 12])

In [235]:
noisy_graph_old_coord = list(nx.get_node_attributes(noisy_graph_old,'coord').values())
noisy_graph_old_coord[:5]

[array([-10.54193644,  78.78266841,  60.68079379]),
 array([ 10.8577999 , -90.23249159,  41.71577212]),
 array([ -7.1834572 , -97.59865898,  20.56452546]),
 array([-32.70926455,  74.32972311, -58.35405963]),
 array([-36.82839016,  12.96707269, -92.06261296])]

In [54]:
#ground_truth_permutation_old

In [257]:
# ground_truth_permutation, noisy_graph = generate_noisy_graph_2(G,len(original_coord),0,10000,0)

In [258]:
#original_coord

In [108]:
noisy_graph_coord = list(nx.get_node_attributes(noisy_graph,'coord').values())
noisy_graph_coord[:5]

[array([-37.2927128 ,  86.0338805 ,  34.74802121]),
 array([-27.01509615,  76.77730558, -58.0984503 ]),
 array([-79.12422892, -49.82797097, -35.44756279]),
 array([ 19.5355599 , -81.85231429,  54.02370355]),
 array([-11.06893283,  79.69635651,  59.37987441])]

In [57]:
ground_truth_permutation

array([ 0,  8, 10, 19,  5,  6,  4, 18,  9, 12, 11,  1,  3, 16,  7, 14, 15,
       13, 17,  2])

In [238]:
original_coord[:5]

[array([ 93.98085132,  11.29476111, -32.24946445]),
 array([-88.83753711, -24.01960081,  39.12736609]),
 array([74.81953047, 18.93880042, 63.58741777]),
 array([  8.42748697, -88.98509987,  44.84004308]),
 array([-29.58511839,   7.77831689, -95.2061897 ])]

In [87]:
original_coord[19]

array([-35.51803649,  86.10340222,  36.39606036])

In [41]:
#ground_truth_permutation_old

In [86]:
# original_coord

In [103]:
shuffled_coord = noisy_graph_coord
random.shuffle(shuffled_coord)

In [105]:
compute_noisy_edges = tri_from_hull(noisy_graph_coord)

In [332]:
noisy_graph_3,noisy_coord_3 = generate_noisy_graph_3(G,len(original_coord),0,10000,0)

nb_outliers:  0


In [333]:
ground_truth_sequence = []

for ar1 in noisy_coord_3:
    
    for i in range(len(noisy_graph_3.nodes)):
        
        if np.mean(ar1) == np.mean(noisy_graph_3.nodes[i]['coord']):
            
            ground_truth_sequence.append(i)
            break
            

In [334]:
ground_truth_sequence

[14, 0, 2, 9, 11, 10, 13, 18, 17, 7, 12, 1, 6, 3, 16, 4, 8, 19, 5, 15]

In [335]:
test_G = noisy_graph_3.copy()

mapping = dict(zip(ground_truth_sequence,list(test_G.nodes)))

test_G = nx.relabel_nodes(test_G, mapping)

In [336]:
test_G.nodes[19]['coord']

array([-35.69382236,  86.21730164,  35.95174465])

In [337]:
noisy_coord_3[19]

array([-35.69382236,  86.21730164,  35.95174465])